In [2]:
import scipy.io as scio  
from skimage import io
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import math
import glob
from keras.models import *

Using TensorFlow backend.


In [3]:
Xception_model=load_model('Xception.h5')
InceptionResNetV_model=load_model('InceptionResNetV2.h5')
InceptionV3_model = load_model('InceptionV3.h5')

KeyboardInterrupt: 

In [26]:
def cell_cut(folders_outline,folders_annatated,Xception_model,InceptionResNetV_model,InceptionV3_model,own_slide):
    img_list_outline= os.listdir('./'+folders_outline)
    img_list_annatated= os.listdir('./'+folders_annatated)
    for m in range(len(img_list_annatated)):
        if img_list_outline[m]=='24_32769_43521(1).npy':
            continue
        data = np.load('./'+folders_outline+'/'+img_list_outline[m])
        data = np.array(data, np.uint8)
        img = cv2.imread('./'+folders_annatated + '/' + os.path.splitext(img_list_outline[m])[0]+'.png')
        k=1
        for n in range(data.shape[2]):
            ret,thresh = cv2.threshold(data[:,:,n],0,255,0)
            image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            for i in range(len(contours)):
                x,y,w,h = cv2.boundingRect(contours[i])
                slide = max(w,h)
                M = cv2.moments(contours[i])
                try:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])
                except ZeroDivisionError:
                    continue
                else:
                    x1 = cx-math.ceil(own_slide/2)
                    y1 = cy-math.ceil(own_slide/2)
                    x2 = cx+math.ceil(own_slide/2)
                    y2 = cy+math.ceil(own_slide/2)
                    if (cx<math.ceil(own_slide/2))or(cy<math.ceil(own_slide/2))or(cx>(513-math.ceil(own_slide/2)))or(cy>(513-math.ceil(own_slide/2))):
                        continue
                    cropImg = img[y1:y2, x1:x2]
                    I = cv2.cvtColor(cropImg,cv2.COLOR_BGR2RGB)
                    I=cv2.resize(I,(139,139))
                    I = I/255
                    I = np.expand_dims(I,axis=0)
                    Xception_preds=Xception_model.predict(I)
                    InceptionV3_preds=InceptionV3_model.predict(I)
                    InceptionResNetV2_preds=InceptionResNetV_model.predict(I)
                    semple = Xception_preds + InceptionV3_preds + InceptionResNetV2_preds
                    preds = np.argmax(semple)
                    preds_pro = max(max(semple))
                    preds_pro = round(preds_pro,2)
                    #preds = np.argmax(preds)
                    if preds==0:
                        cv2.drawContours(img,contours,i,(0,255,0),1)
                    if preds==1:
                        cv2.drawContours(img,contours,i,(0,0,255),1)
                    if preds==2:
                        cv2.drawContours(img,contours,i,(0,255,255),1)
                    font=cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img,str(preds_pro),(cx,cy), font, 0.4,(255,255,255),1)
                    k = k+1
        dir_name='./predict_map'
        if not os.path.exists(dir_name):os.makedirs(dir_name)
        cv2.imwrite(dir_name+'/'+os.path.splitext(img_list_annatated[m])[0]+'_'+str(k)+'.png',img)
        k = k+1

In [27]:
if __name__ == '__main__':
    cell_cut("new_image_classify_v4_67_mask","1290 New Patches",Xception_model,InceptionResNetV_model,InceptionV3_model,70)

KeyboardInterrupt: 